In [ ]:
import os
import sys

# Dynamically add the src directory to the path
sys.path.append(os.path.abspath("../src"))

duckdb_path = "../data/climfinbert_db.duckdb"

In [ ]:
# get all unique values of column FlowName in duckdb
import duckdb

con = duckdb.connect(database=duckdb_path)
cur = con.cursor()
cur.execute("SELECT DISTINCT FlowName FROM my_table")
rows = cur.fetchall()
con.close()

# print all unique values of column FlowName
for row in rows:
    print(row[0])

## Find differences in Senegal for All or None selected donors

### All filters

In [ ]:
from utils.query_duckdb import construct_query, query_duckdb

query = construct_query(
    year_type="single_year",
    selected_year=2018,
    selected_donor_types=["Donor Country", "Multilateral Donor"],
    selected_categories="Adaptation",
    selected_subcategories="Adaptation",
)
print(query)

In [ ]:
from utils.data_operations import reshape_by_type

df_queried = query_duckdb(duckdb_db=duckdb_path, query=query)
df_reshaped = reshape_by_type(df_queried, selected_type="recipients")

In [ ]:
# subset to single country

df_subset = df_reshaped[df_reshaped["CountryCode"] == "BRA"]
df_subset.shape

### None filters

In [ ]:
query = construct_query(
    year_type="single_year",
    selected_year=2018,
    # selected_donor_types=["Donor Country", "Multilateral Donor"],
    selected_categories="Adaptation",
    # selected_subcategories="Adaptation",
)
print(query)

In [ ]:
df_queried = query_duckdb(duckdb_db=duckdb_path, query=query)
df_queried[df_queried["RecipientName"] == "Brazil"].head()

## Test modes

In [ ]:
selected_categories = ["Adaptation", "Mitigation", "Environment"]
query = construct_query(
    year_type="single_year",
    selected_year=2018,
    selected_donor_types=["Donor Country"],
    selected_categories=selected_categories,
    # selected_subcategories="Adaptation",
)

df_queried = query_duckdb(duckdb_db=duckdb_path, query=query)
df_reshaped = reshape_by_type(df_queried, "donors")
df_reshaped.head()

In [ ]:
from utils.data_operations import create_mode_data

create_mode_data(
    df=df_reshaped, map_mode="rio_oecd", selected_categories=selected_categories
)

In [ ]:
create_mode_data(
    df=df_reshaped, map_mode="rio_climfinbert", selected_categories=selected_categories
)

In [ ]:
create_mode_data(
    df=df_reshaped, map_mode="rio_diff", selected_categories=selected_categories
)

## Compare queries & map polygons

In [ ]:
selected_categories = ["Adaptation", "Mitigation", "Environment"]
# selected_subcategories = ["Adaptation", "Mitigation", "Environment"]
query = construct_query(
    year_type="single_year",
    selected_year=2018,
    selected_donor_types=["Private Donor"],
    selected_categories=selected_categories,
    selected_subcategories="Adaptation",
)
df_queried = query_duckdb(duckdb_db=duckdb_path, query=query)
df_reshaped = reshape_by_type(df_queried, "recipients")

df_bra = df_reshaped[df_reshaped["CountryCode"] == "BRA"]

In [ ]:
create_mode_data(
    df=df_bra, map_mode="rio_oecd", selected_categories=selected_categories
)